# This Recomender Systems
## Assignment 1


This project's goal is to build a joke recomender system to make us laugh in
these trying times.

I downloaded the dataset from [this](http://eigentaste.berkeley.edu/dataset/) site.
The dataset is quite wide at 151 rows.



In [66]:
import pandas as pd
import numpy as np
df = pd.read_excel('jester-data-1.xls', header=None)


In [67]:
df.iloc[0:5, 0:10]

,0,1,2,3,4,5,6,7,8,9
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44


In [68]:
df.shape


(24983, 101)

The dataset contains 100 jokes as well the count of the ratings. The user ID is given by the pandas index

In [74]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

In [76]:
np.histogram(df[0], density=True, bins=30)

(array([ 0.02446664,  0.01497268,  0.01437227,  0.01334032,  0.01335908,
         0.01384692,  0.01142652,  0.01435351,  0.00988797,  0.00921251,
         0.00808675,  0.00803046,  0.00756139,  0.00923128,  0.00806799,
         0.0139032 ,  0.05493736,  0.03923293,  0.00904365,  0.00504718,
         0.00536615,  0.00424038,  0.0041841 ,  0.00249545,  0.00217648,
         0.00266431,  0.00227029,  0.00262679,  0.00240163,  0.1379438 ]),
 array([  36.        ,   38.13333333,   40.26666667,   42.4       ,
          44.53333333,   46.66666667,   48.8       ,   50.93333333,
          53.06666667,   55.2       ,   57.33333333,   59.46666667,
          61.6       ,   63.73333333,   65.86666667,   68.        ,
          70.13333333,   72.26666667,   74.4       ,   76.53333333,
          78.66666667,   80.8       ,   82.93333333,   85.06666667,
          87.2       ,   89.33333333,   91.46666667,   93.6       ,
          95.73333333,   97.86666667,  100.        ]))

In [27]:
jokes = df.iloc[:, 1:].replace(99, np.NAN)


In [82]:
def remover(p):
    # Set axis to invisiable
    p.xaxis.axis_line_width = 0.00001
    p.yaxis.axis_line_width = 0.00001
    # Fonts
    p.title.text_font = "times"
    p.title.text_font_style = "normal"
    p.xaxis.axis_label_text_font = 'times'
    p.xaxis.axis_label_text_font_style = 'normal'
    p.yaxis.axis_label_text_font = 'times'
    p.yaxis.axis_label_text_font_style = 'normal'
    # This removes the outline of the graph.
    p.outline_line_color = None
    p.toolbar.logo = None
    p.toolbar_location = None
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    return p

In [88]:
np.histogram(df[0], density=True, bins=10)

(array([ 0.0179372 ,  0.01351544,  0.01188933,  0.00844324,  0.00828689,
         0.0360245 ,  0.00648566,  0.00363998,  0.00237036,  0.04765741]),
 array([  36. ,   42.4,   48.8,   55.2,   61.6,   68. ,   74.4,   80.8,
          87.2,   93.6,  100. ]))

In [106]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file

p1 = figure(title="Frequency of Number of Jokes Rated")

hist, edges = np.histogram(df[0], density=True, bins=30)

#x = np.linspace(-2, 2, 1000)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
    fill_color="skyblue", line_color="grey")
p1.xaxis.axis_label = 'Total Jokes Rated'
p1.yaxis.axis_label = 'Proportion of Responses'
p1 = remover(p1)
show(p1)

We have some complete cases here, however, even if complete cases were not correlated with joke preferences, we still would be thowing out a crazy amount of data for training. 

Since we must train on data that is not a complete case, we need to validate it. Because we must predict NA values for people who have been used as training sets, we need a way to validate.

To do this we can select random ratings to be our test set.

I removed the first column because it was the number of jokes answered.

This is less interesting to us because we don't need to understand how many jokes
were answered. 

In [105]:
p1 = figure(title="Rating Distrobution")
allHist = pd.melt(jokes)
hist, edges = np.histogram(allHist['value'].dropna(), density=True, bins=30)
p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
    fill_color="skyblue", line_color="grey")
p1 = remover(p1)
show(p1)

The data looks like there is a preference for jokes just above average. Also,
more people seem to be fine giving the max negative value indicating they hated
the joke while less give the max positive value. Conversly, there are more people
giving positive values.

---

## Simple Predictions

We are tasked with predicting joke ratings for jokes that users have not yet
rated. There is no way to verify what they will rate. Instead we will have to
validate with a training and test set.

Once we have some strategy, we can apply it to the `NA` values.

Because the NA values are distributed amoung all but a few responders.

In [107]:
jokes.shape

(24983, 100)

In [127]:
np.random.seed(101)
trainTestMask = np.random.choice([True, False], size=jokes.shape, p=[0.7, 0.3])
trainTestMask = pd.DataFrame(trainTestMask)
trainTestMask.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,True,True,True,True,True,False,True,False,False,True,...,True,True,True,True,True,False,True,True,False,True
1,True,True,True,True,False,True,True,True,False,True,...,True,True,True,True,True,False,True,False,True,True
2,True,True,True,True,False,True,True,True,True,True,...,True,True,False,False,False,True,True,False,True,True
3,True,True,True,True,True,True,False,True,True,True,...,True,True,True,True,True,False,False,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,False,True,True,True,False,True,False,True,True,True


This gives us a way to randomly select training and test sets, but we still need to test for NA values.

In [144]:
train = pd.DataFrame(np.where(trainTestMask, jokes, np.NAN))
test = pd.DataFrame(np.where(np.invert(trainTestMask), jokes, np.NAN))

We have our training and test data. We first need to calculate the mean for the total training set and calculate the RMSE on the test set. 

In [158]:
rawMean = train.stack().mean()
rawMean

0.8783613614046262

The raw mean is close to zero. We will replace all na values with this and calculate the RMSE.

In [150]:
testRaw = train.fillna(value=rawMean)

In [166]:
np.sqrt(((pd.melt(test)['value'] - rawMean)**2).mean()) #  This ignores NA values that were present before too.

5.2324418349094142

In [195]:
np.sqrt(((pd.melt(train)['value'] - rawMean)**2).mean()) #  This ignores NA values that were present before too.

5.2373261451264863

So we have a RMSE of 5.23 for the test set and surprisingly, a higher one for the training set. This is a bit weird because we would expect some over fitting. However, given that we are just taking the meanit is unlikely there is a statistical difference. 

## More accuracy

To get more accuracy, we will need to add the column means and row means to the raw mean.

The formula should look like:

$$
pred = RawMean+SpecificColumnMean+SpecificRowMean
$$

Another way to think of it is:

$$
pred_{i,j} = RawMean + RowMean_j + ColMean_i
$$

In [170]:
colMean = train.mean()
rowMean = train.mean(axis=1)

In [188]:
preds = np.full(jokes.shape, rawMean)
preds += colMean
preds =preds + rowMean[:,np.newaxis]

In [190]:
preds = pd.DataFrame(preds)
preds.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-1.885926,-2.646217,-2.469234,-4.251214,-2.334262,-1.162556,-3.202825,-3.425682,-3.344824,-1.531391,...,-0.757232,-1.621067,-0.324863,-1.583458,-1.836889,-1.352553,-1.123196,-2.061757,-2.840812,-1.467379
1,3.927500,3.167209,3.344193,1.562212,3.479165,4.650870,2.610602,2.387745,2.468603,4.282036,...,5.056195,4.192359,5.488563,4.229969,3.976538,4.460873,4.690230,3.751670,2.972614,4.346047
2,8.840250,8.079959,8.256942,6.474962,8.391914,9.563620,7.523351,7.300494,7.381352,9.194785,...,9.968944,9.105109,10.401313,9.142718,8.889287,9.373623,9.602980,8.664419,7.885364,9.258797
3,5.250582,4.490291,4.667274,2.885294,4.802246,5.973952,3.933683,3.710827,3.791684,5.605117,...,6.379276,5.515441,6.811645,5.553050,5.299619,5.783955,6.013312,5.074751,4.295696,5.669129
4,5.170784,4.410493,4.587476,2.805496,4.722449,5.894154,3.853885,3.631029,3.711887,5.525319,...,6.299478,5.435643,6.731847,5.473252,5.219822,5.704157,5.933514,4.994953,4.215898,5.589331


In [193]:
np.sqrt(((preds - test)**2).stack().mean())

4.68875588784186

In [194]:
np.sqrt(((preds - train)**2).stack().mean())

4.6173021500924856

Adding the column and row biases lead to significant imporovement of the RMSE. We now see the training data slightly overfitting when compared to the test data.